# PyTorch Deep Explainer Sample Example

A simple example showing how to explain an MNIST CNN trained using PyTorch with Deep Explainer.

In [1]:
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
# !pip install imblearn
import numpy as np
from model import FraudNet
from load_data import get_dataset, get_dataset_test

from captum.attr import (
    GradientShap,
    DeepLift,
    DeepLiftShap,
    IntegratedGradients,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)

/scratch/scratch4/akashkm99/anaconda/envs/attention/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/scratch4/akashkm99/anaconda/envs/attention/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/scratch4/akashkm99/anaconda/envs/attention/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

ModuleNotFoundError: No module named 'captum'

In [2]:
model = FraudNet(4)
print (model)

FraudNet(
  (embedding_layers): ModuleList(
    (0): Embedding(502, 10)
    (1): Embedding(266, 5)
    (2): Embedding(60, 3)
    (3): Embedding(61, 2)
    (4): Embedding(109, 3)
    (5): Embedding(11625, 20)
    (6): Embedding(4, 2)
    (7): Embedding(3, 2)
    (8): Embedding(106, 2)
    (9): Embedding(5, 2)
  )
  (fc1): Linear(in_features=91, out_features=45, bias=True)
  (fc2): Linear(in_features=45, out_features=22, bias=True)
  (fc3): Linear(in_features=22, out_features=11, bias=True)
  (fc4): Linear(in_features=11, out_features=1, bias=True)
)


In [4]:
# model.load_state_dict(torch.load('continuous_sameWt_eqSample_new_fc_0.3_2_0.333_0.333.th', map_location='cpu'))
model.load_state_dict(torch.load('best_model/sameWt_eqSample_new_fc_0.3_4_0.201_0.202.th', map_location='cpu'))

<All keys matched successfully>

In [5]:
train_loader, valid_loader, test_loader, data = get_dataset(minibatch_size=64, device='cpu')

In [6]:
X_train, X_val, X_test, Y_train, Y_val, Y_test  = data

background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]
background_embed = model.embedding_forward(background).detach().float()
X_test_embed = model.embedding_forward(X_test).detach().float()

In [7]:
model.continuous = True

# Integrated Gradients

In [8]:
ig = IntegratedGradients(model)

In [9]:
attributions, delta = ig.attribute(X_test, torch.zeros(X_test.size()), target=0, return_convergence_delta=True)

RuntimeError: size mismatch, m1: [4314250 x 50], m2: [91 x 45] at /opt/conda/conda-bld/pytorch_1573049304260/work/aten/src/TH/generic/THTensorMath.cpp:197